### MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"


In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts/1', experiment_id='1', lifecycle_stage='active', name='Xgboost Mlops', tags={}>]

In [3]:
from mlflow.entities import ViewType

best_run = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)

In [4]:
# for run in best_run:
#     print(run.data.params)
print(best_run[0].data.params)

{'learning_rate': '0.42649323727247074', 'max_depth': '4', 'min_child_weight': '2.823388102868533', 'objective': 'reg:squarederror', 'reg_alpha': '0.08394020560827088', 'reg_lambda': '0.1989734719817237', 'seed': '42'}


In [5]:
# for run in best_run:
#     best_run_id = run.info.run_id
#     print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']} ")
print(f"run id: {best_run[0].info.run_id}, rmse: {best_run[0].data.metrics['rmse']} ")

run id: 3d3baa9065634314ad797f09905d184a, rmse: 28827.128705456813 


### Interacting with the Model Registry

1. Register the best model with name house-price-regressor
2. Transition the model to "Production"

In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [7]:
model_uri = f"runs:/{best_run[0].info.run_id}/model"
mlflow.register_model(model_uri=model_uri, name="house-price-regressor")

Registered model 'house-price-regressor' already exists. Creating a new version of this model...
2022/08/21 19:05:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: house-price-regressor, version 2
Created version '2' of model 'house-price-regressor'.


<ModelVersion: creation_timestamp=1661108751915, current_stage='None', description='', last_updated_timestamp=1661108751915, name='house-price-regressor', run_id='3d3baa9065634314ad797f09905d184a', run_link='', source='./artifacts/1/3d3baa9065634314ad797f09905d184a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [8]:
model_name = "house-price-regressor"
latest_versions = client.get_latest_versions(name=model_name)

In [9]:
latest_versions

[<ModelVersion: creation_timestamp=1660583052075, current_stage='Production', description='', last_updated_timestamp=1660583111086, name='house-price-regressor', run_id='5654d672c72347ee8beb174c4ca77431', run_link='', source='./artifacts/1/5654d672c72347ee8beb174c4ca77431/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>,
 <ModelVersion: creation_timestamp=1661108751915, current_stage='None', description='', last_updated_timestamp=1661108751915, name='house-price-regressor', run_id='3d3baa9065634314ad797f09905d184a', run_link='', source='./artifacts/1/3d3baa9065634314ad797f09905d184a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>]

In [10]:
model_version = 2
stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=stage
)

<ModelVersion: creation_timestamp=1661108751915, current_stage='Production', description='', last_updated_timestamp=1661108786295, name='house-price-regressor', run_id='3d3baa9065634314ad797f09905d184a', run_link='', source='./artifacts/1/3d3baa9065634314ad797f09905d184a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>